In [79]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# signals
import mne
from scipy.signal import welch, get_window
from scipy.signal.windows import hamming

In [80]:
base_path_data = '/Users/tereza/nishant/atlas/epi_iEEG_atlas/Data'
hup_atlas = sio.loadmat(os.path.join(base_path_data, 'HUP_atlas.mat'))
mni_atlas = sio.loadmat(os.path.join(base_path_data, 'MNI_atlas.mat'))

In [81]:
hup_df = pd.DataFrame(hup_atlas['mni_coords'], columns=['x', 'y', 'z'])
mni_df = pd.DataFrame(mni_atlas['ChannelPosition'], columns=['x', 'y', 'z'])

hup_ts = pd.DataFrame(hup_atlas['wake_clip']) # (12000, 3431) @ x-axis is time steps, y-axis is electrodes
mni_ts = pd.DataFrame(mni_atlas['Data_W']) # (13600, 1765)  @ x-axis is time steps, y-axis is electrodes

hup_patients = pd.DataFrame(hup_atlas['patient_no'])
mni_patients = pd.DataFrame(mni_atlas['Patient'])

hup_patient_ids = np.unique(hup_atlas['patient_no'])
mni_patient_ids = np.unique(mni_atlas['Patient']) 

mni_samp_freq = int(mni_atlas['SamplingFrequency'].flatten()[~np.isnan(mni_atlas['SamplingFrequency'].flatten())][0])
hup_samp_freq = int(hup_atlas['SamplingFrequency'].flatten()[~np.isnan(hup_atlas['SamplingFrequency'].flatten())][0])

hup_patient_total_el_counts = len(hup_atlas['patient_no'])
mni_patient_total_el_counts = len(mni_atlas['Patient'])

hup_patient_numbers = hup_atlas['patient_no'].flatten()
hup_el_to_pat_map_dict = {}
for idx, patient_num in enumerate(hup_patient_numbers):
    hup_el_to_pat_map_dict[idx] = patient_num
hup_idx_map_arr = np.array([patient_num for patient_num in hup_patient_numbers]) # arr equivalent

mni_patient_numbers = mni_atlas['Patient'].flatten()
mni_el_to_pat_map_dict = {}
for idx, patient_num in enumerate(mni_patient_numbers):
    mni_el_to_pat_map_dict[idx] = patient_num
mni_idx_map_arr = np.array([patient_num for patient_num in mni_patient_numbers])

In [82]:
# # for each patient, for each electrode, access electrode data - empty loop
# for patient in hup_patient_ids:
#     patient_el_ids = np.where(hup_idx_map_arr == patient)[0]

#     for idx in patient_el_ids:
#         hup_electrode_data = hup_ts.iloc[:, idx]

# for patient in mni_patient_ids:
#     patient_el_ids = np.where(mni_idx_map_arr == patient)[0]

#     for idx in patient_el_ids:
#         mni_electrode_data = mni_ts.iloc[:, idx]

In [85]:
# # TODO: move to a separate .py script
# def get_norm_psd(iEEGnormal, data_timeS, sampling_frequency=200):
#     """
#     Function to compute normalized power spectral densities for different EEG frequency bands.
    
#     Args:
#     iEEGnormal (DataFrame): A DataFrame to append results to.
#     data_timeS (array): Time domain EEG data.
#     sampling_frequency (int): Sampling frequency of the EEG data.
    
#     Returns:
#     DataFrame: Updated DataFrame with new EEG features.
#     """
    
#     Fs = sampling_frequency
#     window = Fs * 2
#     NFFT = window # frequency resolution; num pts in the FFT
    
#     '''
#     Welch's method (from scipy) divides the signal into overlapping segments,
#     applies a window function to each segment, 
#     computes the FFT of of each windowed segment,
#     averages the FFTs -> reduces noise in the final spectrum
#     Hamming window reduces spectral leakage
#     scaling='density' scales the output to units of power/frequency
#     f - array of frequency points (in Hz) where the PSD is computed
#     data_psd - the PSD values at each frequency point
#     '''
#     # f, data_psd = welch(data_timeS[:Fs*60, :], fs=Fs, window=hamming(window), nfft=NFFT, scaling='density')
#     f, data_psd = welch(data_timeS, fs=Fs, window=hamming(window), nfft=NFFT, scaling='density', noverlap=window//2)
    
#     # filter out noise frequency 57.5Hz to 62.5Hz
#     noise_mask = (f >= 57.5) & (f <= 62.5)
#     f = f[~noise_mask]
#     data_psd = data_psd[:, ~noise_mask]
    
#     '''
#     psd are psd vals from data_psd
#     freqs are the freq points from f
#     freq_range: tuple of (min_freq, max_freq) defining the band
#     trapezodial integration to calculate the area under the PSD curve within that frequency range
#     this area represents the power in that frequency band
#     '''
#     def bandpower(psd, freqs, freq_range):
#         """Calculate power in the given frequency range."""
#         idx = np.logical_and(freqs >= freq_range[0], freqs <= freq_range[1])
#         return np.trapz(psd[idx], freqs[idx])
    
#     bands = {'delta': (1, 4), 'theta': (4, 8), 'alpha': (8, 13), 'beta': (13, 30), 'gamma': (30, 80), 'broad': (1, 80)}
#     #  creates a dictionary where keys are band names and values are their absolute powers
#     # iterate over each key-value pair in the dictionary bands @ band is key, freq_range is value
#     band_powers = {band: bandpower(data_psd, f, freq_range) for band, freq_range in bands.items()}
    
#     # compute log10 transform of band powers -> makes the distribution more normal
#     log_band_powers = {f'{band}log': np.log10(power + 1) for band, power in band_powers.items()}
    
#     # total band power; sums up all the log-transformed powers, needed for normalization
#     total_band_power = np.sum([value for value in log_band_powers.values()])
    
#     # relative band power; normalizes the power in each band by the total power
#     relative_band_powers = {f'{band}Rel': log_band_powers[f'{band}log'] / total_band_power for band in bands}
    
#     # takes the dict of relative band powers and converts it to a single-row df
#     data_to_append = pd.DataFrame([relative_band_powers])
#     data_to_append['broadlog'] = log_band_powers['broadlog']
    
#     # Append to the existing DataFrame
#     iEEGnormal = pd.concat([iEEGnormal, data_to_append], ignore_index=True)
    
#     return iEEGnormal


In [87]:
def get_norm_psd(iEEGnormal, data_timeS, sampling_frequency=200):
    """
    Function to compute normalized power spectral densities for different EEG frequency bands.
    
    Args:
    iEEGnormal (DataFrame): A DataFrame to append results to.
    data_timeS (array): Time domain EEG data for a single electrode (1D array)
    sampling_frequency (int): Sampling frequency of the EEG data.
    
    Returns:
    DataFrame: Updated DataFrame with new EEG features.
    """
    
    Fs = sampling_frequency
    window = Fs * 2
    NFFT = window
    
    # Compute PSD
    f, data_psd = welch(data_timeS, fs=Fs, window=hamming(window), 
                       nfft=NFFT, scaling='density', noverlap=window//2)
    
    # filter out noise frequency 57.5Hz to 62.5Hz
    noise_mask = (f >= 57.5) & (f <= 62.5)
    f = f[~noise_mask]
    # Handle 1D data_psd
    data_psd = data_psd[~noise_mask]
    
    def bandpower(psd, freqs, freq_range):
        """Calculate power in the given frequency range."""
        idx = np.logical_and(freqs >= freq_range[0], freqs <= freq_range[1])
        return np.trapz(psd[idx], freqs[idx])
    
    # Define frequency bands
    bands = {'delta': (1, 4), 'theta': (4, 8), 'alpha': (8, 13), 
             'beta': (13, 30), 'gamma': (30, 80), 'broad': (1, 80)}
    
    # Calculate band powers (using 1D data_psd)
    band_powers = {band: bandpower(data_psd, f, freq_range) 
                  for band, freq_range in bands.items()}
    
    # Compute log transform
    log_band_powers = {f'{band}log': np.log10(power + 1) 
                      for band, power in band_powers.items()}
    
    # Calculate total power
    total_band_power = np.sum([value for value in log_band_powers.values()])
    
    # Calculate relative powers
    relative_band_powers = {f'{band}Rel': log_band_powers[f'{band}log'] / total_band_power 
                          for band in bands}
    
    # Create DataFrame row
    data_to_append = pd.DataFrame([relative_band_powers])
    data_to_append['broadlog'] = log_band_powers['broadlog']
    
    # Append to existing DataFrame
    iEEGnormal = pd.concat([iEEGnormal, data_to_append], ignore_index=True)
    
    return iEEGnormal

In [89]:
hup_iEEGnormal = pd.DataFrame()
mni_iEEGnormal = pd.DataFrame()

# for each patient, for each electrode, compute PSD
for patient in hup_patient_ids:
    print(f"Processing HUP patient {patient}")
    patient_el_ids = np.where(hup_idx_map_arr == patient)[0]
    print(f"Found {len(patient_el_ids)} electrodes")
    
    for idx in patient_el_ids:
        print(f"Processing electrode {idx}")
        hup_electrode_data = hup_ts.iloc[:, idx].values
        print(f"Electrode data shape: {hup_electrode_data.shape}")
        hup_iEEGnormal = get_norm_psd(hup_iEEGnormal, hup_electrode_data)
        print(f"Processed electrode {idx}, DataFrame now has {len(hup_iEEGnormal)} rows")

print("\nFinished HUP processing, starting MNI\n")

for patient in mni_patient_ids:
    print(f"Processing MNI patient {patient}")
    patient_el_ids = np.where(mni_idx_map_arr == patient)[0]
    print(f"Found {len(patient_el_ids)} electrodes")
    
    for idx in patient_el_ids:
        print(f"Processing electrode {idx}")
        mni_electrode_data = mni_ts.iloc[:, idx].values
        print(f"Electrode data shape: {mni_electrode_data.shape}")
        mni_iEEGnormal = get_norm_psd(mni_iEEGnormal, mni_electrode_data)
        print(f"Processed electrode {idx}, DataFrame now has {len(mni_iEEGnormal)} rows")

Processing HUP patient 1
Found 22 electrodes
Processing electrode 0
Electrode data shape: (12000,)
Processed electrode 0, DataFrame now has 1 rows
Processing electrode 1
Electrode data shape: (12000,)
Processed electrode 1, DataFrame now has 2 rows
Processing electrode 2
Electrode data shape: (12000,)
Processed electrode 2, DataFrame now has 3 rows
Processing electrode 3
Electrode data shape: (12000,)
Processed electrode 3, DataFrame now has 4 rows
Processing electrode 4
Electrode data shape: (12000,)
Processed electrode 4, DataFrame now has 5 rows
Processing electrode 5
Electrode data shape: (12000,)
Processed electrode 5, DataFrame now has 6 rows
Processing electrode 6
Electrode data shape: (12000,)
Processed electrode 6, DataFrame now has 7 rows
Processing electrode 7
Electrode data shape: (12000,)
Processed electrode 7, DataFrame now has 8 rows
Processing electrode 8
Electrode data shape: (12000,)
Processed electrode 8, DataFrame now has 9 rows
Processing electrode 9
Electrode data

In [88]:
# hup_iEEGnormal = pd.DataFrame()
# mni_iEEGnormal = pd.DataFrame()

# # for each patient, for each electrode, compute PSD
# for patient in hup_patient_ids:
#     patient_el_ids = np.where(hup_idx_map_arr == patient)[0]
    
#     for idx in patient_el_ids:
#         # retrieves the ts for each electrode as a 1D np arr (expected by get_norm_psd)
#         hup_electrode_data = hup_ts.iloc[:, idx].values 
#         # call the get_norm_psd function for each electrode's ts data
#         hup_iEEGnormal = get_norm_psd(hup_iEEGnormal, hup_electrode_data)

# for patient in mni_patient_ids:
#     patient_el_ids = np.where(mni_idx_map_arr == patient)[0]
    
#     for idx in patient_el_ids:
#         mni_electrode_data = mni_ts.iloc[:, idx].values  
#         mni_iEEGnormal = get_norm_psd(mni_iEEGnormal, mni_electrode_data)  

KeyboardInterrupt: 